## Build Functions for ELT

In [7]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 install -r requirements.txt

In [8]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import modules as f # f is for function

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Download, Transform, and Modeling All in One

In [9]:
# manufactoring suppliers 
# 'lrcs', 'klac', ''

In [30]:
f.predictions('nvda')

NVDA 5m Interval Timestamp: 2024-11-11 15:29:09 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,up,up
kelly_1:2.5,0.171713,0.198031,0.215645,0.10501
prob_up,0.485491,0.374475,0.405597,0.625
prob_static,0.210805,0.301782,0.26441,0.25
prob_down,0.303704,0.323743,0.329993,0.125
precision,0.408367,0.427165,0.439746,0.360721
recall,0.398833,0.422179,0.404669,0.350195
f1,0.403543,0.424658,0.421479,0.35538
support,"[514.0, 514.0, 514.0]","[514.0, 514.0, 514.0]","[514.0, 514.0, 514.0]","[514.0, 514.0, 514.0]"


NVDA 15m Interval Timestamp: 2024-11-11 15:29:18 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,down,down,down
kelly_1:2.5,0.094118,0.204734,0.146341,0.1125
prob_up,0.242836,0.192206,0.289747,0.25
prob_static,0.305168,0.346256,0.305974,0.25
prob_down,0.451996,0.461538,0.404279,0.5
precision,0.352941,0.431953,0.390244,0.366071
recall,0.35503,0.431953,0.378698,0.242604
f1,0.353982,0.431953,0.384384,0.291815
support,"[172.0, 170.0, 169.0]","[172.0, 170.0, 169.0]","[172.0, 170.0, 169.0]","[172.0, 170.0, 169.0]"


NVDA 1h Interval Timestamp: 2024-11-11 15:29:21 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,up,down,up
kelly_1:2.5,0.102242,0.106301,0.105023,0.055963
prob_up,0.314668,0.407032,0.326686,0.5
prob_static,0.360125,0.296424,0.314121,0.25
prob_down,0.325207,0.296544,0.359193,0.25
precision,0.358744,0.361644,0.360731,0.325688
recall,0.37123,0.307692,0.368298,0.331002
f1,0.36488,0.332494,0.364475,0.328324
support,"[429.0, 431.0, 429.0]","[429.0, 431.0, 429.0]","[429.0, 431.0, 429.0]","[429.0, 431.0, 429.0]"


NVDA 1wk Interval Timestamp: 2024-11-11 15:29:34 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,down
kelly_1:2.5,0.051111,0.01087,0.093182,0.066667
prob_up,0.367468,0.287682,0.306559,0.125
prob_static,0.473714,0.483976,0.399007,0.25
prob_down,0.158818,0.228342,0.294433,0.625
precision,0.322222,0.293478,0.352273,0.333333
recall,0.329545,0.306818,0.352273,0.238636
f1,0.325843,0.3,0.352273,0.278146
support,"[88.0, 88.0, 88.0]","[88.0, 88.0, 88.0]","[88.0, 88.0, 88.0]","[88.0, 88.0, 88.0]"


NVDA 1mo Interval Timestamp: 2024-11-11 15:29:36 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,static,static
kelly_1:2.5,0.189474,-0.088889,0.17931,0.183333
prob_up,0.595779,0.408016,0.333123,0.125
prob_static,0.351676,0.323824,0.407997,0.625
prob_down,0.052545,0.26816,0.258881,0.25
precision,0.421053,0.222222,0.413793,0.416667
recall,0.470588,0.117647,0.631579,0.789474
f1,0.444444,0.153846,0.5,0.545455
support,"[17.0, 19.0, 16.0]","[17.0, 19.0, 16.0]","[17.0, 19.0, 16.0]","[17.0, 19.0, 16.0]"


In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')